In [ ]:
!pip install numpy==1.26.4

In [ ]:
# need to restart notebook in order to get numpy 1.x.x after installing it ...
import numpy as np
if int(np.__version__[0]) > 1:
  import os
  os.kill(os.getpid(), 9)

In [ ]:
!pip install timesfm
!pip install u8darts==0.34.0
!pip install pytorch_lightning
!pip install sktime

In [ ]:
import pandas as pd
from darts import TimeSeries
import timesfm
from darts.models import XGBModel
import matplotlib.pyplot as plt

In [ ]:
def wmape(y_pred, y_true):
    """
    Weighted Mean Absolute Percentage Error (WMAPE)

    Args:
        y_true (array-like): Actual values
        y_pred (array-like): Forecasted values

    Returns:
        float: WMAPE value
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))


# Bike sharing dataset

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/lyubomirr/bike-sharing-dataset/refs/heads/main/day.csv')
df

In [ ]:
correlation_matrix = df[['mnth','weekday', 'workingday', 'holiday', 'hum', 'windspeed', 'weathersit', 'temp', 'atemp', 'season', 'cnt']].corr()
correlation_matrix['cnt']

In [ ]:
HORIZON_LENGTH=64
CONTEXT_LENGTH=512

## XGBoost


In [ ]:
def infer_on_xgboost(df, horizon_length, context_length=512, use_covs=True):
  covariates = TimeSeries.from_dataframe(df[['holiday', 'weathersit', 'season', 'mnth', 'atemp', 'windspeed', 'hum']])
  targets = TimeSeries.from_series(df['cnt'])
  ctx = targets[-(context_length+horizon_length):-horizon_length]
  act = targets[-horizon_length:]
  model = XGBModel(lags=20, lags_future_covariates=[0] if use_covs else None)
  model.fit(ctx, future_covariates=covariates if use_covs else None)
  return model.predict(horizon_length), act, ctx

In [ ]:
pred, act, ctx = infer_on_xgboost(df, HORIZON_LENGTH, CONTEXT_LENGTH)
pred_base, _, _ = infer_on_xgboost(df, HORIZON_LENGTH, CONTEXT_LENGTH, False)
print("Base: ", wmape(pred_base.values(), act.values()))
print("Covs: ", wmape(pred.values(), act.values()))

In [ ]:
plt.figure(figsize=(20, 6))

ctx.concatenate(act).plot(label='Цел')
pred_base.plot(label='Базова прогноза', alpha=0.8)
pred.plot(label='Прогноза с допълнителни променливи', alpha=0.8)

plt.title('Прогнозиране на XGBoost с допълнителни променливи')
plt.legend()
plt.show()


## TimesFM


In [ ]:
tfm = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="gpu",
          per_core_batch_size=32,
          horizon_len=HORIZON_LENGTH,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-1.0-200m-pytorch"),
  )

In [ ]:
targets = df['cnt'].to_numpy().reshape(1, -1)

ctx = targets[:, -(CONTEXT_LENGTH + HORIZON_LENGTH):-HORIZON_LENGTH]
act = targets [:, -HORIZON_LENGTH:]

data = df.tail(CONTEXT_LENGTH + HORIZON_LENGTH)
data = data.reset_index()

In [ ]:
raw_forecast, _ = tfm.forecast(ctx, freq=[0])
xreg_timesfm_forecast, _ = tfm.forecast_with_covariates(
      inputs=ctx,
      dynamic_categorical_covariates={
          "holiday": data['holiday'].to_numpy().reshape(1, -1),
          "weathersit": data['weathersit'].to_numpy().reshape(1, -1),
          "season": data['season'].to_numpy().reshape(1, -1),
          "mnth": data['mnth'].to_numpy().reshape(1, -1),
      },
      dynamic_numerical_covariates={
          "atemp":data['atemp'].to_numpy().reshape(1, -1),
          "windspeed": data['windspeed'].to_numpy().reshape(1, -1),
          "hum": data['hum'].to_numpy().reshape(1, -1),

      },
      freq=[0],
      xreg_mode="xreg + timesfm"
  )
timesfm_xreg_forecast, _ = tfm.forecast_with_covariates(
      inputs=ctx,
      dynamic_categorical_covariates={
          "holiday": data['holiday'].to_numpy().reshape(1, -1),
          "weathersit": data['weathersit'].to_numpy().reshape(1, -1),
          "season": data['season'].to_numpy().reshape(1, -1),
          "mnth": data['mnth'].to_numpy().reshape(1, -1),
      },
      dynamic_numerical_covariates={
          "atemp":data['atemp'].to_numpy().reshape(1, -1),
          "windspeed": data['windspeed'].to_numpy().reshape(1, -1),
          "hum": data['hum'].to_numpy().reshape(1, -1),

      },
      freq=[0],
      xreg_mode="timesfm + xreg"
  )

print(wmape(raw_forecast[0].reshape(1, -1), act))
print(wmape(xreg_timesfm_forecast[0].reshape(1, -1), act))
print(wmape(timesfm_xreg_forecast[0].reshape(1, -1), act))

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(targets[0][-(CONTEXT_LENGTH+HORIZON_LENGTH):], label='Цел')
plt.plot(np.concatenate([np.full(CONTEXT_LENGTH, np.nan), raw_forecast[0]]), label='Базова прогноза', alpha=0.8)
plt.plot(np.concatenate([np.full(CONTEXT_LENGTH, np.nan), xreg_timesfm_forecast[0]]), label='Регресия + TimesFM', alpha=0.8)
plt.plot(np.concatenate([np.full(CONTEXT_LENGTH, np.nan), timesfm_xreg_forecast[0]]), label='TimesFM + Регресия', alpha=0.8)
plt.title('Прогнозиране на TimesFM с допълнителни променливи')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(targets[0][-(2*HORIZON_LENGTH):], label='Цел')
plt.plot(np.concatenate([np.full(HORIZON_LENGTH, np.nan), raw_forecast[0]]), label='Базова прогноза')
plt.plot(np.concatenate([np.full(HORIZON_LENGTH, np.nan), xreg_timesfm_forecast[0]]), label='Регресия + TimesFM')
plt.plot(np.concatenate([np.full(HORIZON_LENGTH, np.nan), timesfm_xreg_forecast[0]]), label='TimesFM + Регресия')
plt.title('Прогнозиране на TimesFM с допълнителни променливи (увеличена)')
plt.legend()
plt.grid(True)
plt.show()

# TimesFM v2

In [ ]:
tfm2 = timesfm.TimesFm(
        hparams=timesfm.TimesFmHparams(
            backend="gpu",
            per_core_batch_size=32,
            horizon_len=HORIZON_LENGTH,
            num_layers=50,
            use_positional_embedding=False,
            context_len=2048,
        ),
        checkpoint=timesfm.TimesFmCheckpoint(
            huggingface_repo_id="google/timesfm-2.0-500m-pytorch"),
  )

In [ ]:
targets = df['cnt'].to_numpy().reshape(1, -1)

ctx = targets[:, -(CONTEXT_LENGTH + HORIZON_LENGTH):-HORIZON_LENGTH]
act = targets [:, -HORIZON_LENGTH:]

data = df.tail(CONTEXT_LENGTH + HORIZON_LENGTH)
data = data.reset_index()

In [ ]:
raw_forecast2, _ = tfm2.forecast(ctx, freq=[0])
xreg_timesfm_forecast2, _ = tfm2.forecast_with_covariates(
      inputs=ctx,
      dynamic_categorical_covariates={
          "holiday": data['holiday'].to_numpy().reshape(1, -1),
          "weathersit": data['weathersit'].to_numpy().reshape(1, -1),
          "season": data['season'].to_numpy().reshape(1, -1),
          "mnth": data['mnth'].to_numpy().reshape(1, -1),
      },
      dynamic_numerical_covariates={
          "atemp":data['atemp'].to_numpy().reshape(1, -1),
          "windspeed": data['windspeed'].to_numpy().reshape(1, -1),
          "hum": data['hum'].to_numpy().reshape(1, -1),

      },
      freq=[0],
      xreg_mode="xreg + timesfm"
  )
timesfm_xreg_forecast2, _ = tfm2.forecast_with_covariates(
      inputs=ctx,
      dynamic_categorical_covariates={
          "holiday": data['holiday'].to_numpy().reshape(1, -1),
          "weathersit": data['weathersit'].to_numpy().reshape(1, -1),
          "season": data['season'].to_numpy().reshape(1, -1),
          "mnth": data['mnth'].to_numpy().reshape(1, -1),
      },
      dynamic_numerical_covariates={
          "atemp":data['atemp'].to_numpy().reshape(1, -1),
          "windspeed": data['windspeed'].to_numpy().reshape(1, -1),
          "hum": data['hum'].to_numpy().reshape(1, -1),

      },
      freq=[0],
      xreg_mode="timesfm + xreg"
  )

print(wmape(raw_forecast2[0].reshape(1, -1), act))
print(wmape(xreg_timesfm_forecast2[0].reshape(1, -1), act))
print(wmape(timesfm_xreg_forecast2[0].reshape(1, -1), act))

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(targets[0][-(CONTEXT_LENGTH+HORIZON_LENGTH):], label='Цел')
plt.plot(np.concatenate([np.full(CONTEXT_LENGTH, np.nan), raw_forecast2[0]]), label='Базова прогноза', alpha=0.8)
plt.plot(np.concatenate([np.full(CONTEXT_LENGTH, np.nan), xreg_timesfm_forecast2[0]]), label='Регресия + TimesFM v2', alpha=0.8)
plt.plot(np.concatenate([np.full(CONTEXT_LENGTH, np.nan), timesfm_xreg_forecast2[0]]), label='TimesFM v2 + Регресия', alpha=0.8)
plt.title('Прогнозиране на TimesFM v2 с допълнителни променливи')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(targets[0][-(2*HORIZON_LENGTH):], label='Цел')
plt.plot(np.concatenate([np.full(HORIZON_LENGTH, np.nan), raw_forecast2[0]]), label='Базова прогноза')
plt.plot(np.concatenate([np.full(HORIZON_LENGTH, np.nan), xreg_timesfm_forecast2[0]]), label='Регресия + TimesFM')
plt.plot(np.concatenate([np.full(HORIZON_LENGTH, np.nan), timesfm_xreg_forecast2[0]]), label='TimesFM + Регресия')
plt.title('Прогнозиране на TimesFM v2 с допълнителни променливи (увеличена)')
plt.legend()
plt.grid(True)
plt.show()

## AutoARIMA

In [ ]:
from sktime.forecasting.statsforecast import StatsForecastAutoARIMA

In [ ]:
targets = df['cnt'].to_numpy()

ctx = targets[-(CONTEXT_LENGTH+HORIZON_LENGTH):-HORIZON_LENGTH]
act = targets [-HORIZON_LENGTH:]

data = df.tail(CONTEXT_LENGTH + HORIZON_LENGTH)
data = data.reset_index()

In [ ]:
fit_covs = data[['holiday', 'weathersit', 'season', 'mnth', 'atemp', 'windspeed', 'hum']].to_numpy(dtype=float)[:-HORIZON_LENGTH, :]
pred_covs = data[['holiday', 'weathersit', 'season', 'mnth', 'atemp', 'windspeed', 'hum']].to_numpy(dtype=float)[-HORIZON_LENGTH:, :]

In [ ]:
arima = StatsForecastAutoARIMA()

arima.fit(ctx.reshape(-1, 1))
arima_base = arima.predict(fh=np.arange(1, HORIZON_LENGTH+1))

arima.reset()

arima.fit(ctx.reshape(-1, 1), X=fit_covs)
arima_pred = arima.predict(fh=np.arange(1, HORIZON_LENGTH+1), X=pred_covs)

In [ ]:
print("Base: ", wmape(arima_base, act))
print("Cov: ", wmape(arima_pred, act))

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(targets[-(CONTEXT_LENGTH+HORIZON_LENGTH):], label='Цел')
plt.plot(np.concatenate([np.full(512, np.nan), arima_base.reshape(-1)]), label='Базова прогноза', alpha=0.8)
plt.plot(np.concatenate([np.full(512, np.nan), arima_pred.reshape(-1)]), label='Прогноза с допълнителни променливи', alpha=0.8)
plt.title('Прогнозиране на ARIMA с допълнителни променливи')
plt.legend()
plt.grid(True)
plt.show()
